# Creating an Observable from a Simulation 
When running a refinement, MDMC makes quantitative comparisons between properties measured experimentally and calculated from MD simulations: within MDMC each of these properties is an `Observable`. Examples of these observables are the dynamic structure factor, $S(Q,\omega)$, and the pair distribution function, $G(r)$.

Within a refinement, each `Observable` is calculated from MD automatically. However it is also useful to be able to calculate an `Observable` from an MD simulation and plot it, which is demonstrated in this tutorial.

This tutorial requires matplotlib to be installed:

In [1]:
try:
    %matplotlib notebook
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
except ImportError:
    %pip install matplotlib
    try:
        %matplotlib notebook
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D
    except ImportError:
        print('Please restart the kernel so that matplotlib can be imported.')

## Running a simulation

Below we setup and run a simulation of liquid argon so that we have a `Trajectory` from which to calculate an `Observable`.

**As this is minimizing, equilibrating, and running a production run, this should take ~3 minutes to execute**. Minimization lowers the potential energy of the simulated system by adjusting atomic positions. Running a simulation with `equilibration` set to `True` will not record a `Trajectory`, and since we specified neither a thermostat or barastat for the simulation a Berendsen thermostat will be applied for the duration of the equilibration. It will not be present for the second run, when the `Trajectory` is recorded.

In [2]:
from MDMC.MD import *
import numpy as np

# Build universe
universe = Universe(dimensions=38.4441)
Ar = Atom('Ar', charge=0.)
# Fill box with 1000 atoms, and hence atoms per AA^-3 density = 0.0176
universe.fill(Ar, num_struc_units=1000)
print('Universe contains {} Ar atoms'.format(len(universe.atoms)))
Ar_dispersion = Dispersion(universe,
                           (Ar.atom_type, Ar.atom_type),
                           cutoff=8.,
                           vdw_tail_correction=True,
                           function=LennardJones(1.0243, 3.36))

# MD Engine setup
simulation = Simulation(universe,
                        engine="lammps",
                        time_step=10.0,
                        temperature=120.,
                        traj_step=25)

# Energy Minimization and equilibration
simulation.minimize(n_steps=5000)
simulation.run(n_steps=20000, equilibration=True)
simulation.run(12025)
trajectory = simulation.trajectory

Universe created with:
  Dimensions       [38.44, 38.44, 38.44]
  Force field                       None
  Number of atoms                      0

Universe contains 1000 Ar atoms
LAMMPS output is captured by PyLammps wrapper
LAMMPS output is captured by PyLammps wrapper


Simulation created with lammps engine and settings:
  temperature  120.0



### 3D surface plotting

Below we define a simple function for plotting 3D surfaces, which we will use for plotting the observables.

In [3]:
def plot_surface(x, y, z):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    X, Y = np.meshgrid(x, y)
    surf = ax.plot_surface(X, Y, z)
    plt.show()

## Calculating $S(Q,\omega)$

To see which observables can be calculated, use the `help` function to look at the observables module documentation:

In [4]:
from MDMC.trajectory_analysis import observables
help(observables)

Help on package MDMC.trajectory_analysis.observables in MDMC.trajectory_analysis:

NAME
    MDMC.trajectory_analysis.observables - Modules calculating observables from molecular dynamics trajectories

DESCRIPTION
    Observables (/ indicates alias)
    -----------
    DynamicStructureFactor / SQw
    CoherentDynamicStructureFactor / SQwCoherent / SQwCoh / SQw_coh
    IncoherentDynamicStructureFactor / SQwIncoherentSQwIncoh / SQw_incoh
    IntermediateScatteringFunction / FQt
    IncoherentIntermediateScatteringFunction / FQtIncoherentFQtIncoh / FQt_incoh
    CoherentIntermediateScatteringFunction / FQtCoherent / FQtCoh / FQt_coh
    PDF / PairDistributionFunction
    
    Examples
    --------
    Observables can be instantiated using the names above. For instance an SQw
    observable can be instantiated using either of the aliases:
    
    .. code-block:: python
    
        from MDMC.trajectory_analysis import observables
        sqw = observables.SQw()                     # This l

As is mentioned in the Examples section, each observable may have one or more aliases.  So both `DynamicStructureFactor` or `SQw` can be used for calculating $S(Q,\omega)$.

Some of the details given below are specific to calculating the dynamic structure factor, however the general method can be followed for any observable.

Argon has comparable scattering lengths for both coherent and incoherent scattering, so the total scattering has clear contributions from each.  For the purposes of this tutorial we will simply calculate $S(Q,\omega)_{coh}$ and compare it with experimental data; the exercise of calculating the incoherent and total scattering is left as an exercise for the reader.

In [5]:
sqw_coh = observables.SQwCoh()

The dynamic structure factor has two independent variables, the scattering wavevector, $Q$, and $E$ (or $\hbar\omega$) is the energy transfer.

The size of the simulation box defines the spacing (and therefore the lower bound) of $Q$ values; however any Q values can be specified for calculating $S(Q,\omega)$ and MDMC will simply calculate it for valid values.

In [6]:
# Using the arange function from NumPy - this creates an uniform array of floats with a
# start (0.42), stop (3.8) and step (0.3) specified below
Q = np.arange(0.42, 3.8, 0.3)
print('Q will be calculated for values of: {}'.format(Q))

Q will be calculated for values of: [0.42 0.72 1.02 1.32 1.62 1.92 2.22 2.52 2.82 3.12 3.42 3.72]


The total duration of the trajectory defines the smallest spacing of the $E$ values.  If $E$ values are not specified, MDMC will simply calculate the $E$ values with this spacing.  However for this setting there will only be two points in the trajectory contributing to each energy bin and the statistics will be poor (**should take ~30s to execute**):

In [7]:
# Set the Q values for the Q independent variable
sqw_coh.independent_variables = {'Q':Q}

# Calculate the coherent dynamic structure factor
sqw_coh.calculate_from_MD(trajectory)

# Use the plotting function defined above to plot the result
plot_surface(sqw_coh.E, sqw_coh.Q, sqw_coh.SQw[0])

<IPython.core.display.Javascript object>

Therefore it is strongly recommended that you specify the $E$ values for calculating the dynamic structure factor.

The allowed values of $E$ (in meV) are determined by trajectory times.  To aid in calculating these, the `calulate_E` method 
can be used, which requires the time step of the trajectory (in fs) and the total number of E values to be calculated.  The number of E values should be changed to improve the statistics, but somewhere in the range of 1/4 to 1/8 of the total number of trajectory configurations (i.e. the number of times for which a configuration was recorded) will be probably reasonable:

In [8]:
# This subtracts the time of the zeroeth step from the time of the first step
# It should be equal to the time_step * traj_step passed to Simulation
trajectory_timestep = trajectory.times[1] - trajectory.times[0]
print('Trajectory time step: {} fs'.format(trajectory_timestep))

n_configurations = len(trajectory.times)
print('Number of trajectory configurations: {}'.format(n_configurations))

Trajectory time step: 250.0 fs
Number of trajectory configurations: 481


In [9]:
E = sqw_coh.calculate_E((n_configurations - 1) / 6, trajectory_timestep)
print('Calculated E values are: \n{} meV'.format(E))

Calculated E values are: 
[0.         0.10339169 0.20678338 0.31017508 0.41356677 0.51695846
 0.62035015 0.72374185 0.82713354 0.93052523 1.03391692 1.13730862
 1.24070031 1.344092   1.44748369 1.55087539 1.65426708 1.75765877
 1.86105046 1.96444216 2.06783385 2.17122554 2.27461723 2.37800893
 2.48140062 2.58479231 2.688184   2.79157569 2.89496739 2.99835908
 3.10175077 3.20514246 3.30853416 3.41192585 3.51531754 3.61870923
 3.72210093 3.82549262 3.92888431 4.032276   4.1356677  4.23905939
 4.34245108 4.44584277 4.54923447 4.65262616 4.75601785 4.85940954
 4.96280124 5.06619293 5.16958462 5.27297631 5.376368   5.4797597
 5.58315139 5.68654308 5.78993477 5.89332647 5.99671816 6.10010985
 6.20350154 6.30689324 6.41028493 6.51367662 6.61706831 6.72046001
 6.8238517  6.92724339 7.03063508 7.13402678 7.23741847 7.34081016
 7.44420185 7.54759355 7.65098524 7.75437693 7.85776862 7.96116031
 8.06455201 8.1679437 ] meV


Of course if the $E$ resolution required is lower (i.e. step between the `E` values can be larger), the number of E values specified can be reduced:

In [10]:
print('A reduced E resolution: \n{} meV'.format(sqw_coh.calculate_E((n_configurations - 1) / 24, trajectory_timestep)))

A reduced E resolution: 
[0.         0.41356677 0.82713354 1.24070031 1.65426708 2.06783385
 2.48140062 2.89496739 3.30853416 3.72210093 4.1356677  4.54923447
 4.96280124 5.376368   5.78993477 6.20350154 6.61706831 7.03063508
 7.44420185 7.85776862] meV


Now we can use `Q` and `E` to set the indepedendent variables of `sqw_coh`:

In [11]:
sqw_coh.independent_variables = {'E':E,
                                 'Q':Q}

Using this binning, recalculate the dynamic structure factor, which is smoother due to better statistics **This calculation should take ~30s.**:

In [12]:
sqw_coh.calculate_from_MD(trajectory)
plot_surface(sqw_coh.E, sqw_coh.Q, sqw_coh.SQw[0])

<IPython.core.display.Javascript object>

We can compare this with experimental data measured using quasi-elastic neutron scattering (QENS) data (specifically here the van Well et al. Phys. Rev. A **31** 3391 (1985) data, which the authors corrected for instrument resolution) by reading the data into an SQwCoh object:

In [13]:
# Create a second SQwCoh object, this time called sqw_exp
sqw_exp = observables.SQwCoh()

# Read the data from the file ./data/Well_s_q_omega_Ar_data.xml using the `xml_SQw` reader
sqw_exp.read_from_file('xml_SQw', './data/Well_s_q_omega_Ar_data.xml')

The independent and dependent variables are read into the same attributes (`sqw_coh.E`, `sqw_exp.Q`, and `sqw_exp.SQw`) as if they had been calculated from MD (as in `sqw_coh` above).

For many QENS datasets (if not most) the instrument resolution has not been removed, and to compare with MD simulation this requires the simulation output to be convolution with the instrument resolution. One way this can be done is using: the optional `energy_resolution`:   

In [14]:
# energy_resolution is the FWHM of the instrumentation's resolution function expressed in ueV (micro eV).
# It is used to smooth the calculated S(Q,w) with a Gaussian window in order to match the values obtained experimentally.
sqw_coh.calculate_from_MD(trajectory, energy_resolution={'gaussian': 1.0e3})

plot_surface(sqw_coh.E, sqw_coh.Q, sqw_coh.SQw[0])

<IPython.core.display.Javascript object>

## Other observables

The same applies for the observables, such as the pair distribution function $G(r)$.  To determine how to calculate another observables from an MD trajectory, please see the associated help documentation:

In [15]:
help(observables.PDF.calculate_from_MD)

Help on function calculate_from_MD in module MDMC.trajectory_analysis.observables.pdf:

calculate_from_MD(self, MD_input: Union[MDMC.trajectory_analysis.trajectory.Trajectory, List[MDMC.trajectory_analysis.trajectory.Trajectory]], **settings)
    Calculate the pair distribution function, :math:`G(r)`` from a
    ``Trajectory``
    
    The total pair distribution function (``pdf.PDF``) has the form:
    
    .. math::
    
        G(r) = \sum_{i,j}^{N_{elements}} c_ic_jb_ib_j(g_{ij}(r) - 1)
    
    where :math:`c_i` is the number concentration of element :math:`i`,
    :math:`b_i` is the (coherent) scattering length of element :math:`i`,
    and the partial pair distribution, :math:`g_{ij}`, is:
    
    .. math::
    
        g_{ij}(r) = \frac{h_{ij}(r)}{4 \pi r^2 \rho_{j} \Delta{r}}
    
    where :math:`h_{ij}`` is the histogram of distances of :math:`j` element
    atoms around atoms of element :math:`i`, with bins of size
    :math:`\Delta{r}`, and :math:`\rho_{j}` is the number 